In [4]:
import os
os.chdir('/Users/tanvi/Downloads/')

In [7]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier 
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [8]:
#pip install lightgbm

In [9]:
train=pd.read_csv('train_bank analytics.csv')
test=pd.read_csv('test_bank analytics.csv')
data=pd.concat([train,test], axis=0, ignore_index=True, sort=False)

In [11]:
(train.shape),test.shape

((164309, 14), (109541, 13))

In [12]:
train.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [14]:
print(data.isnull().sum())

Loan_ID                         0
Loan_Amount_Requested           0
Length_Employed             12307
Home_Owner                  42060
Annual_Income               42000
Income_Verified                 0
Purpose_Of_Loan                 0
Debt_To_Income                  0
Inquiries_Last_6Mo              0
Months_Since_Deliquency    147238
Number_Open_Accounts            0
Total_Accounts                  0
Gender                          0
Interest_Rate              109541
dtype: int64


In [16]:
data.describe()

,Loan_ID,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,2.738500e+05,2.318500e+05,273850.000000,273850.000000,126612.000000,273850.000000,273850.000000,164309.000000
mean,1.013693e+07,7.339279e+04,17.215901,0.784543,34.103395,11.186025,25.067975,2.158951
std,7.905383e+04,5.852984e+04,7.845967,1.036816,21.750307,4.973663,11.589677,0.738364
min,1.000000e+07,3.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,1.006846e+07,4.500000e+04,11.360000,0.000000,16.000000,8.000000,17.000000,2.000000
50%,1.013693e+07,6.300000e+04,16.850000,0.000000,31.000000,10.000000,23.000000,2.000000
75%,1.020539e+07,8.900000e+04,22.780000,1.000000,49.000000,14.000000,32.000000,3.000000
max,1.027385e+07,7.500000e+06,39.990000,8.000000,180.000000,84.000000,156.000000,3.000000


In [ ]:
#### data preprocessing

In [17]:
#****filling nans with mean 
#Label encoding / one hot encoding for category columns
#log transformation wherever found neccessary
## replacing , with space


In [18]:
data['Loan_Amount_Requested']=data['Loan_Amount_Requested'].str.replace("," ,'')

data['Loan_Amount_Requested']=data['Loan_Amount_Requested'].astype(int)

data['Loan_Amount_Requested']=np.log(data['Loan_Amount_Requested'])

data.describe()


,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,2.738500e+05,273850.000000,2.318500e+05,273850.000000,273850.000000,126612.000000,273850.000000,273850.000000,164309.000000
mean,1.013693e+07,9.379342,7.339279e+04,17.215901,0.784543,34.103395,11.186025,25.067975,2.158951
std,7.905383e+04,0.668087,5.852984e+04,7.845967,1.036816,21.750307,4.973663,11.589677,0.738364
min,1.000000e+07,6.214608,3.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,1.006846e+07,8.987197,4.500000e+04,11.360000,0.000000,16.000000,8.000000,17.000000,2.000000
50%,1.013693e+07,9.392662,6.300000e+04,16.850000,0.000000,31.000000,10.000000,23.000000,2.000000
75%,1.020539e+07,9.903488,8.900000e+04,22.780000,1.000000,49.000000,14.000000,32.000000,3.000000
max,1.027385e+07,10.463103,7.500000e+06,39.990000,8.000000,180.000000,84.000000,156.000000,3.000000


In [19]:
# considerable amount of data is missing for the column Months_Since_Deliquency
data['Months_Since_Deliquency'].describe()
data['Months_Since_Deliquency']=np.log(data['Months_Since_Deliquency'], where = data['Months_Since_Deliquency']!=0)
data.describe()

,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,2.738500e+05,273850.000000,2.318500e+05,273850.000000,273850.000000,126612.000000,273850.000000,273850.000000,164309.000000
mean,1.013693e+07,9.379342,7.339279e+04,17.215901,0.784543,3.242998,11.186025,25.067975,2.158951
std,7.905383e+04,0.668087,5.852984e+04,7.845967,1.036816,0.873018,4.973663,11.589677,0.738364
min,1.000000e+07,6.214608,3.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,1.006846e+07,8.987197,4.500000e+04,11.360000,0.000000,2.772589,8.000000,17.000000,2.000000
50%,1.013693e+07,9.392662,6.300000e+04,16.850000,0.000000,3.433987,10.000000,23.000000,2.000000
75%,1.020539e+07,9.903488,8.900000e+04,22.780000,1.000000,3.891820,14.000000,32.000000,3.000000
max,1.027385e+07,10.463103,7.500000e+06,39.990000,8.000000,5.192957,84.000000,156.000000,3.000000


In [21]:
data['Annual_Income'].describe()
data['Annual_Income']=np.log(data['Annual_Income'])

In [22]:
data['Number_Open_Accounts']=np.log(data['Number_Open_Accounts'], where = data['Number_Open_Accounts']!=0)

In [23]:
data['Total_Accounts']=np.log(data['Total_Accounts'])
data.describe()

,Loan_ID,Loan_Amount_Requested,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,2.738500e+05,273850.000000,231850.000000,273850.000000,273850.000000,126612.000000,273849.000000,273850.000000,164309.000000
mean,1.013693e+07,9.379342,11.058225,17.215901,0.784543,3.242998,2.317625,3.107733,2.158951
std,7.905383e+04,0.668087,0.520884,7.845967,1.036816,0.873018,0.452407,0.498548,0.738364
min,1.000000e+07,6.214608,8.006368,0.000000,0.000000,0.000000,0.000000,0.693147,1.000000
25%,1.006846e+07,8.987197,10.714418,11.360000,0.000000,2.772589,2.079442,2.833213,2.000000
50%,1.013693e+07,9.392662,11.050890,16.850000,0.000000,3.433987,2.302585,3.135494,2.000000
75%,1.020539e+07,9.903488,11.396392,22.780000,1.000000,3.891820,2.639057,3.465736,3.000000
max,1.027385e+07,10.463103,15.830414,39.990000,8.000000,5.192957,11.695247,5.049856,3.000000


In [24]:

data['Length_Employed']=data['Length_Employed'].str.replace('years', '')

data['Length_Employed']=data['Length_Employed'].replace('1 year', 1)

data['Length_Employed']=data['Length_Employed'].replace('10+ ', 11)

data['Length_Employed']=data['Length_Employed'].replace('< 1 year', 0)

data['Length_Employed'].fillna(21, inplace=True)

data['Length_Employed']=data['Length_Employed'].astype(int)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273850 entries, 0 to 273849
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Loan_ID                  273850 non-null  int64  
 1   Loan_Amount_Requested    273850 non-null  float64
 2   Length_Employed          273850 non-null  int64  
 3   Home_Owner               231790 non-null  object 
 4   Annual_Income            231850 non-null  float64
 5   Income_Verified          273850 non-null  object 
 6   Purpose_Of_Loan          273850 non-null  object 
 7   Debt_To_Income           273850 non-null  float64
 8   Inquiries_Last_6Mo       273850 non-null  int64  
 9   Months_Since_Deliquency  126612 non-null  float64
 10  Number_Open_Accounts     273849 non-null  float64
 11  Total_Accounts           273850 non-null  float64
 12  Gender                   273850 non-null  object 
 13  Interest_Rate            164309 non-null  float64
dtypes: f

In [25]:
data.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,8.853665,0,Rent,11.127263,not verified,car,18.37,0,NaN,2.197225,2.639057,Female,1.0
1,10000002,10.308953,4,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,2.833213,2.484907,3.178054,Female,3.0
2,10000003,10.115570,7,Mortgage,11.232767,VERIFIED - income source,debt_consolidation,15.88,0,NaN,2.484907,2.772589,Male,3.0
3,10000004,9.680344,0,NaN,10.935960,VERIFIED - income source,debt_consolidation,14.34,3,NaN,2.772589,3.091042,Male,3.0
4,10000005,9.740969,8,Own,11.472103,VERIFIED - income source,debt_consolidation,22.17,1,NaN,2.944439,3.401197,Female,1.0


In [26]:
label_encode=LabelEncoder()

data['Gender']=label_encode.fit_transform(data['Gender'])

In [27]:
data

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,8.853665,0,Rent,11.127263,not verified,car,18.37,0,NaN,2.197225,2.639057,0,1.0
1,10000002,10.308953,4,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,2.833213,2.484907,3.178054,0,3.0
2,10000003,10.115570,7,Mortgage,11.232767,VERIFIED - income source,debt_consolidation,15.88,0,NaN,2.484907,2.772589,1,3.0
3,10000004,9.680344,0,NaN,10.935960,VERIFIED - income source,debt_consolidation,14.34,3,NaN,2.772589,3.091042,1,3.0
4,10000005,9.740969,8,Own,11.472103,VERIFIED - income source,debt_consolidation,22.17,1,NaN,2.944439,3.401197,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273845,10273846,9.615805,9,Mortgage,11.695247,VERIFIED - income,other,8.15,2,3.465736,2.564949,4.043051,1,NaN
273846,10273847,9.169518,11,Mortgage,11.736069,not verified,credit_card,9.08,1,2.833213,2.197225,3.526361,1,NaN
273847,10273848,7.600902,5,Rent,10.541386,not verified,credit_card,11.60,0,NaN,2.302585,2.397895,1,NaN
273848,10273849,7.661527,6,Rent,10.308953,not verified,other,24.88,0,NaN,1.609438,1.791759,1,NaN


In [28]:
#Label encoding opposite to the number of records found for that sub category 
data['Purpose_Of_Loan']=data['Purpose_Of_Loan'].map({'debt_consolidation':1, 'credit_card':2,
                                                           'home_improvement':3, 'other':4,
                                                           'major_purchase':5, 'small_business':6,
                                                           'car':7, 'medical':8, 
                                                           'moving':9, 'vacation':10,
                                                           'wedding':11, 'house':12,
                                                           'renewable_energy':13, 'educational':14})

In [30]:
print(data.isnull().sum())

Loan_ID                         0
Loan_Amount_Requested           0
Length_Employed                 0
Home_Owner                  42060
Annual_Income               42000
Income_Verified                 0
Purpose_Of_Loan                 0
Debt_To_Income                  0
Inquiries_Last_6Mo              0
Months_Since_Deliquency    147238
Number_Open_Accounts            1
Total_Accounts                  0
Gender                          0
Interest_Rate              109541
dtype: int64


In [31]:
data.columns

Index(['Loan_ID', 'Loan_Amount_Requested', 'Length_Employed', 'Home_Owner',
       'Annual_Income', 'Income_Verified', 'Purpose_Of_Loan', 'Debt_To_Income',
       'Inquiries_Last_6Mo', 'Months_Since_Deliquency', 'Number_Open_Accounts',
       'Total_Accounts', 'Gender', 'Interest_Rate'],
      dtype='object')

In [32]:
# home owner missing have been replaced by unknown label
data['Home_Owner'].fillna('unknown', inplace=True)


In [37]:
data1=pd.get_dummies(data, columns=['Home_Owner', 'Income_Verified'], drop_first=True)

In [39]:
data1.shape

(273850, 19)

In [40]:
## open account ratio
data1_try = data1.copy()
data1_try['account_ratio'] = data1_try['Number_Open_Accounts']/data1_try['Total_Accounts']
data1_try2 = data1_try.copy()
data1_try2['closed_account_ratio'] = 1 - data1_try['account_ratio']





#Creating X and Y

We have different datasets for different features combo . We chose data1_try2 because it gave the best f1 score

In [43]:
Dataset_test=data1_try2[data1_try2['Interest_Rate'].isnull()==True]
Dataset_train=data1_try2[data1_try2['Interest_Rate'].isnull()!=True]

In [47]:
list1 = ['Interest_Rate', 'Loan_ID']

X=Dataset_train.drop(list1, axis=1)
Y=Dataset_train['Interest_Rate']

In [48]:
X

,Loan_Amount_Requested,Length_Employed,Annual_Income,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Home_Owner_unknown,Income_Verified_VERIFIED - income source,Income_Verified_not verified,account_ratio,closed_account_ratio
0,8.853665,0,11.127263,7,18.37,0,NaN,2.197225,2.639057,0,0,0,0,1,0,0,1,0.832579,0.167421
1,10.308953,4,NaN,1,14.93,0,2.833213,2.484907,3.178054,0,0,0,0,0,0,0,0,0.781896,0.218104
2,10.115570,7,11.232767,1,15.88,0,NaN,2.484907,2.772589,1,0,0,0,0,0,1,0,0.896241,0.103759
3,9.680344,0,10.935960,1,14.34,3,NaN,2.772589,3.091042,1,0,0,0,0,1,1,0,0.896975,0.103025
4,9.740969,8,11.472103,1,22.17,1,NaN,2.944439,3.401197,0,0,0,1,0,0,1,0,0.865707,0.134293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164304,9.939096,11,NaN,2,13.56,0,NaN,2.564949,2.772589,1,0,0,0,1,0,1,0,0.925110,0.074890
164305,8.699515,8,NaN,6,9.12,0,3.583519,2.564949,3.401197,1,0,0,1,0,0,1,0,0.754131,0.245869
164306,9.915910,11,NaN,2,18.17,1,NaN,2.833213,3.401197,1,0,0,0,0,0,0,0,0.833005,0.166995
164307,9.210340,4,NaN,5,18.75,1,3.871201,2.772589,4.127134,0,0,0,0,1,0,0,1,0.671795,0.328205


In [53]:
X=X.iloc[:, :].values
Y=Y.iloc[:].values

imputer=SimpleImputer(missing_values=np.nan, strategy='mean')

X[:, 2:3]=imputer.fit_transform(X[:, 2:3])

X[:, 6:7]=imputer.fit_transform(X[:, 6:7])

pd.DataFrame(X).isnull().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [54]:
print(X)

[[ 8.85366543  0.         11.12726298 ...  1.          0.83257933
   0.16742067]
 [10.30895266  4.         11.05778819 ...  0.          0.78189571
   0.21810429]
 [10.11557016  7.         11.23276702 ...  0.          0.89624063
   0.10375937]
 ...
 [ 9.91591007 11.         11.05778819 ...  0.          0.83300468
   0.16699532]
 [ 9.21034037  4.         11.05778819 ...  1.          0.67179512
   0.32820488]
 [ 9.13237883 11.         11.05778819 ...  0.          0.73248676
   0.26751324]]


In [55]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1,random_state=0)

In [56]:
lg_2=LGBMClassifier(boosting_type='gbdt', max_depth=9, learning_rate=0.1, objective='multiclass', random_state=100,  
                  n_estimators=406 ,reg_alpha=0, reg_lambda=1, n_jobs=-1)


In [58]:
lg_2

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=406, n_jobs=-1, num_leaves=31,
               objective='multiclass', random_state=100, reg_alpha=0,
               reg_lambda=1, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [59]:
lg_2.fit(X_train,Y_train)                                  
Y_predict=lg_2.predict(X_test)

In [60]:
print(f1_score(Y_test,Y_predict, average='weighted'))

0.540487463223255


In [61]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

def classifier_cross_val_score( classifier, features, labels, cv_fold, scoring_strategy ):
    
    
    cv_score = cross_val_score(classifier, features, labels, cv=cv_fold, scoring=scoring_strategy)
    
#     print("Actual CV SCORE LIST : "+str(cv_score))

    return np.mean(cv_score)

In [62]:
cv_fold = 5
classifier_cross_val_score(lg_2, X, Y, cv_fold, scoring_strategy='f1_weighted')


0.5344977777435517

In [63]:
lg_2.feature_importances_

array([6224, 1896, 5048, 2205, 5214, 1314, 3211, 1879, 3133,  317,    0,
         16,  220,  439,  272,  399,  518, 2956, 1269], dtype=int32)

In [64]:
Dataset_test.shape

(109541, 21)

In [73]:
X_main_test=Dataset_test.drop(['Interest_Rate', 'Loan_ID'], axis=1)

X_main_test=X_main_test.iloc[:, :].values

X_main_test[:, 2:3]=imputer.fit_transform(X_main_test[:, 2:3])

X_main_test[:, 6:7]=imputer.fit_transform(X_main_test[:, 6:7])

pd.DataFrame(X_test).isnull().sum()

Y_test_predict=lg_2.predict(X_main_test)

sample_submission=pd.read_csv('sample_submission_HSqiq1Q.csv')
sample_submission['Interest_Rate']=Y_test_predict

sample_submission.to_csv('model_submission.csv', index=False)

In [70]:
X_main_test

array([[10.22194128, 11.        , 11.76756768, ...,  0.        ,
         0.63767307,  0.36232693],
       [10.16585182, 11.        , 11.60823564, ...,  1.        ,
         0.57144404,  0.42855596],
       [ 8.71193727,  0.        , 11.22524339, ...,  0.        ,
         0.53724357,  0.46275643],
       ...,
       [ 7.60090246,  5.        , 10.54138626, ...,  1.        ,
         0.96025257,  0.03974743],
       [ 7.66152708,  6.        , 10.30895266, ...,  1.        ,
         0.8982444 ,  0.1017556 ],
       [ 9.61580548,  2.        , 11.8277362 , ...,  1.        ,
         0.73395163,  0.26604837]])

In [69]:
sample_submission

,Loan_ID,Interest_Rate
0,10164310,2.0
1,10164311,2.0
2,10164312,3.0
3,10164313,2.0
4,10164314,2.0
...,...,...
109536,10273846,3.0
109537,10273847,2.0
109538,10273848,2.0
109539,10273849,3.0
